In [1]:

import os

In [2]:
%pwd

'd:\\Resume_Projects\\Text-Summarizer-Project\\research'

In [3]:

os.chdir("../")

In [4]:
%pwd

'd:\\Resume_Projects\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelTrainerConfig:
    root_dir: str
    data_path: str
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int
    max_train_samples: int = None  # ✅ added here


In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
            max_train_samples=params.max_train_samples  # <-- FIXED this line
        )

        return model_trainer_config


In [8]:
import torch
print(torch.__version__)


2.4.1+cpu


In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\gunas\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-06 19:00:13,997: INFO: config: PyTorch version 2.4.1 available.]


In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        # Load dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Reduce training samples if max_train_samples is specified
        if hasattr(self.config, "max_train_samples") and self.config.max_train_samples is not None:
            dataset_samsum_pt["train"] = dataset_samsum_pt["train"].select(range(min(self.config.max_train_samples, len(dataset_samsum_pt["train"]))))
            dataset_samsum_pt["validation"] = dataset_samsum_pt["validation"].select(range(min(20, len(dataset_samsum_pt["validation"]))))  # optional: small val set

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        # Save model and tokenizer
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [11]:
try:
    config = ConfigurationManager()
    trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=trainer_config)
    model_trainer.train()
except Exception as e:
    raise e


[2025-04-06 19:00:19,030: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-06 19:00:19,039: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-06 19:00:19,044: INFO: common: created directory at: artifacts]
[2025-04-06 19:00:19,048: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\gunas\anaconda3\envs\textS\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\gunas\AppData\Local\Temp\ipykernel_18388\2151857438.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
100%|██████████| 3/3 [08:26<00:00, 171.90s/it]c:\Users\gunas\anaconda3\envs\textS\lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the c

{'train_runtime': 531.2218, 'train_samples_per_second': 0.282, 'train_steps_per_second': 0.006, 'train_loss': 2.985084851582845, 'epoch': 2.48}
